### API 키 불러오기

In [1]:
import os
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

In [4]:
import sys
from typing import List, Union, Optional, Any, Dict
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate, load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

### Get Stage Analyzer Prompt

In [5]:
stage_analyzer_inception_prompt = load_prompt("./templates/stage_analyzer_inception_prompt_template.json")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)
stage_analyzer_chain = LLMChain(
    llm=llm,
    prompt=stage_analyzer_inception_prompt, 
    verbose=False, 
    output_key="stage_number")

### Get User Response candidates Generation Prompt

In [6]:
user_response_prompt = load_prompt("./templates/user_response_prompt.json")
# 랭체인 모델 선언, 랭체인은 언어모델과 프롬프트로 구성됩니다.
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
user_response_chain = LLMChain(
    llm=llm,
    prompt=user_response_prompt, 
    verbose=False, # 과정을 출력할지
    output_key="user_responses"
)

### Load wine database json

In [7]:
df = pd.read_json('./data/unified_wine_data.json', encoding='utf-8', lines=True)

In [8]:
df.head()

,url,site_name,price,name,en_name,img_url,body,acidity,tannin,sweetness,alcohol,wine_type,country,grape,rating,pickup_location,vivino_link
0,https://www.winenara.com/shop/product/product_...,winenara,49000,모노폴 클라시코,Monopole Classico,https://www.winenara.com/uploads/product/550/1...,3,,,-1,-1,화이트,스페인,,3.8,,https://www.vivino.com/monopole-la-rioja-blanc...
1,https://www.winenara.com/shop/product/product_...,winenara,32000,슐럼베르거 로제 스페셜 브뤼,Schlumberger Rose Special Brut,https://www.winenara.com/uploads/product/550/d...,3,,,-1,-1,스파클링,독일,,3.8,,https://www.vivino.com/schlumberger-spring-edi...
2,https://www.winenara.com/shop/product/product_...,winenara,50000,SET)페데럴리스트 샤르도네 원통 패키지,SET)THE FEDERALIST CHARDONNAY,https://www.winenara.com/uploads/product/550/d...,3,,,-1,-1,화이트,미국,,3.7,,https://www.vivino.com/federalist-chardonnay-m...
3,https://www.winenara.com/shop/product/product_...,winenara,55000,베니카 트레 비니스,VENICA TRE VIGNIS,https://www.winenara.com/uploads/product/550/c...,4,,,-1,-1,화이트,이탈리아,,3.9,,https://www.vivino.com/US-CA/en/venica-venica-...
4,https://www.winenara.com/shop/product/product_...,winenara,24900,SET)빌라엠비앙코 + 글라스2개 윈터패키지,SET)VILLA M Bianco + GLASS WINTER PACKAGE,https://www.winenara.com/uploads/product/550/a...,-1,,,4,-1,디저트,이탈리아,,3.9,,https://www.vivino.com/villa-m-bianco/w/1774733


### Prepare Langchain Tool

#### Tool1: Wine database 1

In [9]:
loader =DataFrameLoader(data_frame=df, page_content_column='name')
docs = loader.load()
embeddings = OpenAIEmbeddings()

아래는 wine database1에 metadata_field Attribute이다. 아래를 기준으로 서치를 진행하게 된다.

In [10]:
metadata_field_info = [
    AttributeInfo(
        name="body",
        description="1-5 rating for the body of wine",
        type="int",
    ),
    AttributeInfo(
        name="sweetness",
        description="1-5 rating for the sweetness of wine",
        type="int",
    ),
    AttributeInfo(
        name="alcohol",
        description="1-5 rating for the alcohol of wine",
        type="int",
    ),
    AttributeInfo(
        name="price",
        description="The price of the wine",
        type="int",
    ),
    AttributeInfo(
        name="rating", 
        description="1-5 rating for the wine", 
        type="float"
    ),
    AttributeInfo(
        name="wine_type", 
        description="The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'", 
        type="string"
    ),
    AttributeInfo(
        name="country", 
        description="The country of wine. It can be '기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주'", 
        type="float"
    ),
]

In [11]:
vectorstore = Chroma.from_documents(docs, embeddings)
document_content_description = "Database of a wine"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=False
)  # Added missing closing parenthesis


In [12]:
retriever.get_relevant_documents('{"wine_type":"레드", "body": "lt 3 gt 0"}') # gt means greater than, lt means less than, eq means equal to

/Users/minsu/miniconda/envs/chatwine/lib/python3.9/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[Document(page_content='바 독 피노누아', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03P976', 'site_name': 'winenara', 'price': 29000, 'en_name': 'BAR DOG PINOT NOIR', 'img_url': 'https://www.winenara.com/uploads/product/550/1936_detail_084.png', 'body': 2, 'acidity': '', 'tannin': '', 'sweetness': -1, 'alcohol': -1, 'wine_type': '레드', 'country': '미국', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/US-CA/en/bar-dog-pinot-noir/w/7129723'}),
 Document(page_content='루이라뚜르 피노누아', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03H965', 'site_name': 'winenara', 'price': 52000, 'en_name': 'LOUIS LATOUR PINOT NOIR', 'img_url': 'https://www.winenara.com/uploads/product/550/493_detail_025.png', 'body': 2, 'acidity': '', 'tannin': '', 'sweetness': -1, 'alcohol': -1, 'wine_type': '레드', 'country': '프랑스', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/G

#### Tool2: Search specific wine with url

In [13]:
def search_with_url(query):
    return SeleniumURLLoader(urls=[query]).load()

#### Tool3: Wine database 2

In [14]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

#### Tool4: Search in Google

In [15]:
search = SerpAPIWrapper()

In [16]:
tools = [
    Tool(
        name="Wine database",
        func=retriever.get_relevant_documents,
        description="""
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- body: 1-5 rating int for the body of wine. You have to specify greater than or less than. For example, if you want to search for wines with a body rating of less than 3, enter 'body: gt 0 lt 3'
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt20000'
- rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
- wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
- name: The name of wine. 입력할 때는 '와인 이름은 "비냐 조잘" 입니다' 이런 식으로 입력해주세요.
"""
    ),
    Tool(
        name = "Search specific wine with url",
        func=search_with_url,
        description="Search specific wine with url. Query must be url"
    ),
    Tool(
        name = "Wine database 2",
        func=index.query,
        description="Database about the wines in wine store. You can use this tool if you're having trouble getting information from the wine database tool above. Query must be in String"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to ask with search. Search in English only."
    ),
]

In [17]:
template = """
Your role is a chatbot that asks customers questions about wine and makes recommendations.
Never forget your name is "이우선".
Keep your responses in short length to retain the user's attention. 
Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the user a chance to respond.
Responses should be in Korean.

Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
이우선: the final response to the user

You must respond according to the conversation stage within the triple backticks and conversation history within in '======'.

Current conversation stage: 
```{conversation_stage}```

Conversation history: 
=======
{conversation_history}
=======

Last user saying: {input}
{agent_scratchpad}
"""

conversation_stages_dict = {
    "1": "Start: Start the conversation by introducing yourself. Be polite and respectful while maintaining a professional tone of conversation.",
    "2": "Analyze: Identify the user's preferences in order to make wine recommendations. Ask questions to understand the preferences of your users in order to make wine recommendations. Ask only one question at a time. The wine database tool is not available here.",
    "3": "Recommendation: Recommend the right wine based on the user's preferences identified. Recommendations must be limited to wines in wine database, and you can use tools to do this. After making a wine recommendation, it asks if the user likes the wine you recommended.",
    "4": "After recommendation: If the user likes the wine you recommended, provides a link and image of wine. Otherwise, it takes you back to the recommendation stage.",
    "5": "Close: When you're done, say goodbye to the user.",
    "6": "Question and Answering: This is where you answer the user's questions. To answer user question, you can use the search tool or the wine database tool.",
    "7": "Not in the given steps: This step is for when none of the steps between 1 and 6 apply.",
}

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        stage_number = kwargs.pop("stage_number")
        kwargs["conversation_stage"] = conversation_stages_dict[stage_number]
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps", "conversation_history", "stage_number"]
)

In [18]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "이우선: " in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("이우선: ")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

### Define Langchain Agent

In [19]:
from langchain.callbacks.streaming_stdout_final_only import StreamingStdOutCallbackHandler, FinalStreamingStdOutCallbackHandler
import queue

In [20]:
class CustomStreamingStdOutCallbackHandler(FinalStreamingStdOutCallbackHandler):
    """Callback handler for streaming in agents.
    Only works with agents using LLMs that support streaming.

    The output will be streamed until "END" is reached.
    """

    def __init__(
        self,
        *,
        answer_prefix_tokens: Optional[List[str]] = None,
        end_prefix_tokens: str = "<END",
        strip_tokens: bool = True,
        stream_prefix: bool = False,
        sender: str
    ) -> None:
        """Instantiate EofStreamingStdOutCallbackHandler.

        Args:
            answer_prefix_tokens: Token sequence that prefixes the anwer.
                Default is ["Final", "Answer", ":"]
            end_of_file_token: Token that signals end of file.
                Default is "END"
            strip_tokens: Ignore white spaces and new lines when comparing
                answer_prefix_tokens to last tokens? (to determine if answer has been
                reached)
            stream_prefix: Should answer prefix itself also be streamed?
        """
        super().__init__(answer_prefix_tokens=answer_prefix_tokens, strip_tokens=strip_tokens, stream_prefix=stream_prefix)
        self.end_prefix_tokens = end_prefix_tokens
        self.end_reached = False
        self.sender = sender

    def append_to_last_tokens(self, token: str) -> None:
        self.last_tokens.append(token)
        self.last_tokens_stripped.append(token.strip())
        if len(self.last_tokens) > 5:
            self.last_tokens.pop(0)
            self.last_tokens_stripped.pop(0)

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Run when LLM starts running."""
        self.answer_reached = False
        self.end_reached = False

    def check_if_answer_reached(self) -> bool:
        if self.strip_tokens:
            return ''.join(self.last_tokens_stripped) in self.answer_prefix_tokens_stripped
        else:
            unfied_last_tokens = ''.join(self.last_tokens)
            try:
                unfied_last_tokens.index(self.answer_prefix_tokens)
                return True
            except:
                return False
            
    def check_if_end_reached(self) -> bool:
        if self.strip_tokens:
            return ''.join(self.last_tokens_stripped) in self.answer_prefix_tokens_stripped
        else:
            unfied_last_tokens = ''.join(self.last_tokens)
            # print(unfied_last_tokens)
            try:
                unfied_last_tokens.index(self.end_prefix_tokens)
                self.sender[1] = True
                return True
            except:
                return False

    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        """Run on new LLM token. Only available when streaming is enabled."""
        # Remember the last n tokens, where n = len(answer_prefix_tokens)
        self.append_to_last_tokens(token)
        # Check if the last n tokens match the answer_prefix_tokens list ...
        if not self.answer_reached and self.check_if_answer_reached():
            self.answer_reached = True
            if self.stream_prefix:
                for t in self.last_tokens:
                    sys.stdout.write(t)
                sys.stdout.flush()
            return
        
        if not self.end_reached and self.check_if_end_reached():
            self.end_reached = True

        # ... if yes, then print tokens from now on, unless EOF has been reached
        # print(self.answer_reached, self.end_reached)
        if self.end_reached:
            pass
        elif self.answer_reached:
            if self.last_tokens[-2] == ":":
                pass
            else:
                # global agent_string
                # agent_string += self.last_tokens[-2]
                self.sender[0] += self.last_tokens[-2]
                
                # sys.stdout.write(self.last_tokens[-2])
                # sys.stdout.flush()

In [21]:
# llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-4', temperature=0.0), prompt=prompt, verbose=True,)
# llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-4', temperature=0.0, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]), prompt=prompt, verbose=True,)
class UnifiedAgent:
    def __init__(self):
        sender_for_gradio = ["", False]

        tools = [
            Tool(
                name="Wine database",
                func=retriever.get_relevant_documents,
                description="""
        Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
        You can search wines with the following attributes:
        - body: 1-5 rating int for the body of wine. You have to specify greater than or less than. For example, if you want to search for wines with a body rating of less than 3, enter 'body: gt 0 lt 3'
        - price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt20000'
        - rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
        - wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
        - name: The name of wine. 입력할 때는 '와인 이름은 "비냐 조잘" 입니다' 이런 식으로 입력해주세요.
        """
            ),
            Tool(
                name = "Search specific wine with url",
                func=search_with_url,
                description="Search specific wine with url. Query must be url"
            ),
            Tool(
                name = "Wine database 2",
                func=index.query,
                description="Database about the wines in wine store. You can use this tool if you're having trouble getting information from the wine database tool above. Query must be in String"
            ),
            Tool(
                name = "Search",
                func=search.run,
                description="Useful for when you need to ask with search. Search in English only."
            )
        ]

        llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0, streaming=True, callbacks=[CustomStreamingStdOutCallbackHandler(answer_prefix_tokens='이우선:', end_prefix_tokens='<END', strip_tokens=False, sender=sender_for_gradio)]), prompt=prompt, verbose=False,)

        tool_names = [tool.name for tool in tools]
        agent = LLMSingleActionAgent(
            llm_chain=llm_chain, 
            output_parser=output_parser,
            stop=["\nObservation:"], 
            allowed_tools=tool_names
        )
        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=False)

        self.agent_executor = agent_executor
        self.sender = sender_for_gradio


class UnifiedChain:
    def __init__(self):
        stage_analyzer_inception_prompt = load_prompt("./templates/stage_analyzer_inception_prompt_template.json")
        llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)
        stage_analyzer_chain = LLMChain(
            llm=llm,
            prompt=stage_analyzer_inception_prompt, 
            verbose=False, 
            output_key="stage_number")
        
        user_response_prompt = load_prompt("./templates/user_response_prompt.json")
        # 랭체인 모델 선언, 랭체인은 언어모델과 프롬프트로 구성됩니다.
        llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
        user_response_chain = LLMChain(
            llm=llm,
            prompt=user_response_prompt, 
            verbose=False, # 과정을 출력할지
            output_key="user_responses"
        )

        self.stage_analyzer_chain = stage_analyzer_chain
        self.user_response_chain = user_response_chain

In [22]:
# agent_executor.run()

### Start Conversation

In [23]:
# user_response = ""
# conversation_history, pre_conversation_history = """""", """"""
# stage_history = ""

In [24]:
# stage_number = stage_analyzer_chain.run({'conversation_history': conversation_history, 'stage_history': stage_history})
# print(f'stage_number: {stage_number}')
# stage_history += stage_number if stage_history == "" else ", " + stage_number
# response = agent_executor.run({'input':user_response, 'conversation_history': pre_conversation_history, 'stage_number': stage_number})
# # for chunk in  agent_executor.run({'input':user_response, 'conversation_history': pre_conversation_history, 'stage_number': stage_number}):
    
# conversation_history += "이우선: " + response + "\n"

In [25]:
# user_response = input("User: ")
# pre_conversation_history = conversation_history
# conversation_history += f"User: {user_response} <END_OF_TURN>\n"

In [26]:
# for i in conversation_history.split('\n'):
#     print(i)

### Gradio

간단하게 웹 구성을 테스트하는 gradio이다. 개선해야할 점이 많지만 맛보기로 올려보았다.

In [27]:
# import gradio as gr
# import threading
# import time

In [28]:
# stage_description = ""
# for key, value in conversation_stages_dict.items():
#     stage_description += f"{key}.{value}\n"

# with gr.Blocks(css='#chatbot .overflow-y-auto{height:750px}') as demo:
 
#     with gr.Row():
#         gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0 auto;">
#             <div>
#                 <h1>ChatWine</h1>
#             </div>
#             <p style="margin-bottom: 10px; font-size: 94%">
#                 LinkedIn <a href="https://www.linkedin.com/company/audrey-ai/about/">Audrey.ai</a>
#             </p>
#         </div>""")
 
#     chatbot = gr.Chatbot()
#     msg = gr.Textbox(label='User input')
#     samples = [["이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래?"], ["입문자에게 좋은 와인을 추천해줄래?"], ["보르도와 부르고뉴 와인의 차이점은 뭐야?"]]
#     user_response_examples = gr.Dataset(samples=samples, components=[msg], type="index")
#     stage_history = gr.Textbox(value="stage history: ", interactive=False, label='stage history')
#     submit_btn = gr.Button("전송")
#     clear_btn = gr.ClearButton([msg, chatbot])
#     stage_info = gr.Textbox(value=stage_description, interactive=False, label='stage description')

    # def agent_run(agent_exec, inp):
    #     agent_exec(inp)


#     def load_example(example_id):
#         return samples[example_id][0]
    
#     def submit_user_response(user_response, chat_history):
#         return "", chat_history + [[user_response, None]]

#     def answer(user_response, chat_history, stage_history):
#         chat_history = chat_history or []
#         stage_history = stage_history or ""
#         pre_conversation_history = ""
#         for idx, chat in enumerate(chat_history):
#             pre_conversation_history += f"User: {chat[0]} <END_OF_TURN>\n"
#             pre_conversation_history += f"이우선: {chat[1]} <END_OF_TURN>\n"
#         conversation_history = pre_conversation_history + f"User: {user_response} <END_OF_TURN>\n"

#         stage_number = stage_analyzer_chain.run({'conversation_history': conversation_history, 'stage_history': stage_history})
#         stage_number = stage_number[-1]
#         stage_history += stage_number if stage_history == "stage history: " else ", " + stage_number

#         global agent_string
#         agent_string = ""
#         t = threading.Thread(target = agent_run, args=(agent_executor, {'input':user_response, 'conversation_history': pre_conversation_history, 'stage_number': stage_number}))
#         t.start()

#         while(t.is_alive()):
#             time.sleep(0.05)
#             chat_history[-1][1] = agent_string
#             yield "", chat_history, stage_history, gr.Dataset.update(samples=samples)

#         t.join()

#         conversation_history += "이우선: " + agent_string + "\n"
#         agent_string = agent_string.split('<END_OF_TURN>')[0]
        
#         user_response_examples = []
        
#         for user_response_example in user_response_chain.run({'conversation_history': conversation_history}).split('|'):
#             user_response_examples.append([user_response_example])
#         samples = user_response_examples

#         print(chat_history)
 
#         yield "", chat_history, stage_history, gr.Dataset.update(samples=samples)

#     def clear(*args):
#         samples = [["이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래?"], ["입문자에게 좋은 와인을 추천해줄래?"], ["보르도와 부르고뉴 와인의 차이점은 뭐야?"]]
#         return gr.Dataset.update(samples=samples), "stage history: "

#     clear_btn.click(fn=clear, inputs=[user_response_examples, stage_history], outputs=[user_response_examples, stage_history])
#     user_response_examples.click(load_example, inputs=[user_response_examples], outputs=[msg])      
#     submit_btn.click(submit_user_response, [msg, chatbot], [msg, chatbot]).then(answer, [msg, chatbot, stage_history], [msg, chatbot, stage_history, user_response_examples])
#     msg.submit(submit_user_response, [msg, chatbot], [msg, chatbot]).then(answer, [msg, chatbot, stage_history], [msg, chatbot, stage_history, user_response_examples])

In [29]:
# demo.queue()
# demo.launch()
# # demo.launch(share=True)

In [30]:
gr.close_all()

NameError: name 'gr' is not defined

In [31]:
import gradio as gr
import threading
import time

/Users/minsu/miniconda/envs/chatwine/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
with gr.Blocks(css='#chatbot .overflow-y-auto{height:750px}') as demo:
    unified_chain = UnifiedChain()
    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>ChatWine</h1>
            </div>
            <p style="margin-bottom: 10px; font-size: 94%">
                LinkedIn <a href="https://www.linkedin.com/company/audrey-ai/about/">Audrey.ai</a>
            </p>
        </div>""")
    
    chatbot = gr.Chatbot()

    with gr.Row():
        with gr.Column(scale=0.85):
            msg = gr.Textbox()
        with gr.Column(scale=0.15, min_width=0):
            submit_btn = gr.Button("전송")

    user_response_examples = gr.Dataset(samples=[["이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래?"], ["입문자에게 좋은 와인을 추천해줄래?"], ["보르도와 부르고뉴 와인의 차이점은 뭐야?"]], components=[msg], type="index")
    clear_btn = gr.ClearButton([msg, chatbot])

    dev_mod = True
    cur_stage = gr.Textbox(visible=dev_mod, interactive=False, label='current_stage')
    stage_hist = gr.Textbox(visible=dev_mod, value="stage history: ", interactive=False, label='stage history')
    chat_hist = gr.Textbox(visible=dev_mod, interactive=False, label='chatting_history')
    response_examples_text = gr.Textbox(visible=dev_mod, interactive=False, value="이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래?|입문자에게 좋은 와인을 추천해줄래?|보르도와 부르고뉴 와인의 차이점은 뭐야?", label='response_examples')


    def load_example(response_text, input_idx):
        response_examples = []
        for user_response_example in response_text.split('|'):
            response_examples.append([user_response_example])
        return response_examples[input_idx][0]

    def agent_run(agent_exec, inp, sender):
        sender[0] = ""
        agent_exec.run(inp)

    def user_chat(user_message, chat_history_list, chat_history):
        return (chat_history_list + [[user_message, None]], chat_history + f"User: {user_message} <END_OF_TURN>\n", [])

    def bot_stage_pred(chat_history, stage_history):
        stage_number = unified_chain.stage_analyzer_chain.run({'conversation_history': chat_history, 'stage_history': stage_history})
        stage_number = stage_number[-1]
        stage_history += stage_number if stage_history == "stage history: " else ", " + stage_number

        return stage_number, stage_history

    def bot_chat(user_response, chat_history, chat_history_list, current_stage): # stream output by yielding
        unified_agent = UnifiedAgent()
        t = threading.Thread(target = agent_run, args=(unified_agent.agent_executor, {'input':user_response, 'conversation_history': chat_history, 'stage_number': current_stage}, unified_agent.sender))
        t.start()

        while(t.is_alive()):
            time.sleep(0.05)
            response = unified_agent.sender[0]
            chat_history_list[-1][1] = response
            yield chat_history_list, chat_history + f"이우선: {response}\n"

        t.join()
        response = unified_agent.sender[0]
        chat_history_list[-1][1] = response
        return chat_history_list, chat_history + f"이우선: {response}\n"

    def bot_response_pred(chat_history):
        response_examples = []
        out = unified_chain.user_response_chain.run({'conversation_history': chat_history})
        for user_response_example in out.split('|'):
            response_examples.append([user_response_example])
        return [response_examples, out, ""]

    msg.submit(
        user_chat, [msg, chatbot, chat_hist], [chatbot, chat_hist, user_response_examples], queue=False
    ).then(
        bot_stage_pred, [chat_hist, stage_hist], [cur_stage, stage_hist], queue=False
    ).then(
        bot_chat, [msg, chat_hist, chatbot, cur_stage], [chatbot, chat_hist]
    ).then(
        bot_response_pred, chat_hist, [user_response_examples, response_examples_text, msg]
    )

    submit_btn.click(
        user_chat, [msg, chatbot, chat_hist], [chatbot, chat_hist, user_response_examples], queue=False
    ).then(
        bot_stage_pred, [chat_hist, stage_hist], [cur_stage, stage_hist], queue=False
    ).then(
        bot_chat, [msg, chat_hist, chatbot, cur_stage], [chatbot, chat_hist]
    ).then(
        bot_response_pred, chat_hist, [user_response_examples, response_examples_text, msg]
    )
    clear_btn.click(lambda: None, None, chatbot, queue=False)
    user_response_examples.click(load_example, inputs=[response_examples_text, user_response_examples], outputs=[msg])

demo.queue()
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://fe7b7b8592d734b4c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
















 보르
 보르도
 보르도와
 보르도와 부
 보르도와 부르
 보르도와 부르
 보르도와 부르고
 보르도와 부르고뉴 와
 보르도와 부르고뉴 와인
 보르도와 부르고뉴 와인의
 보르도와 부르고뉴 와인의 차
 보르도와 부르고뉴 와인의 차이
 보르도와 부르고뉴 와인의 차이점
 보르도와 부르고뉴 와인의 차이점은
 보르도와 부르고뉴 와인의 차이점은 지
 보르도와 부르고뉴 와인의 차이점은 지역
 보르도와 부르고뉴 와인의 차이점은 지역과 포
 보르도와 부르고뉴 와인의 차이점은 지역과 포
 보르도와 부르고뉴 와인의 차이점은 지역과 포도
 보르도와 부르고뉴 와인의 차이점은 지역과 포도
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다.
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다.
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다.
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다.
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑스
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑스 보
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑스 보르도
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑스 보르도
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑스 보르도 지역
 보르도와 부르고뉴 와인의 차이점은 지역과 포도 품종입니다. 보르도 와인은 프랑스 보르도 지역에서
 보